# Part 1: Function to Parse WhatsApp Chat for Task Completion

In [15]:
import re
from datetime import datetime, date  # Importing date to use date.today()

def parse_task_completion_and_additions(file_path, start_date, end_date):
	"""
	Read the WhatsApp file path, handle the date format, convert start and end dates to determine object,
	use regex to capture the date, time, user, and message. Additionally, capture user additions.
	"""
	# Set to store users who have sent 'done', 'dn', or 'تم'
	done_users = set()
	
	# Set to store users who were added or who added someone
	added_users = set()
	added_by_others = set()
	requested_to_join_users = set()

	# Read the WhatsApp file
	with open(file_path, 'r', encoding='utf-8') as f:
		lines = f.readlines()

	# format (dd.mm.yyyy)
	date_format = '%d.%m.%Y'

	# Convert the start and end dates to datetime objects
	start_date = datetime.strptime(start_date, date_format).date()
	end_date = datetime.strptime(end_date, date_format).date()

	# Regex patterns
	# message_pattern = re.compile(r'\[(\d{1,2}\.\d{1,2}\.\d{4}) (\d{2}:\d{2}:\d{2})\] (?:~\s*|)(.*?): (.*)')
	# added_pattern = re.compile(r'~\s*(.*?) was added')
	# added_by_pattern = re.compile(r'~\s*(.*?) added ~\s*(.*)')
	message_pattern = re.compile(r'\[(\d{1,2}\.\d{1,2}\.\d{4})(?:,)? (\d{2}:\d{2}:\d{2})\] (?:~\s*|)(.*?): (.*)')
	added_pattern = re.compile(r'~\s*(.*?) was added')
	added_by_pattern = re.compile(r'~\s*(.*?) added ~\s*(.*)')
	join_from_community_pattern = re.compile(r'\[(\d{1,2}\.\d{1,2}\.\d{4})(?:,)? (\d{2}:\d{2}:\d{2})\] (?:~\s*|)(.*?): (.*) joined from the community')
	requested_to_join_pattern = re.compile(r'\[(\d{1,2}\.\d{1,2}\.\d{4})(?:,)? (\d{2}:\d{2}:\d{2})\] ~\s*(.*?): ~\s*(.*) requested to join')


	current_user = None
	current_message = ""

	for line in lines:
		match = message_pattern.match(line)
		join_match = join_from_community_pattern.match(line)
		request_match = requested_to_join_pattern.match(line)
		
		if match:
			# If there's a new match, process the previous message before resetting
			if current_user and current_message:
				# Process the previous message for task completion
				cleaned_message = current_message.strip().lower()
				if ('done' in cleaned_message or 
					'dn' in cleaned_message or 
					'✅' in cleaned_message or
					'done✅' in cleaned_message or
					'done' in cleaned_message or
					'تم ' in cleaned_message or
					'ضن' in cleaned_message or
					re.search(r'\bتم\b', cleaned_message) or 
					cleaned_message == 'تم' or 
					'تم' in cleaned_message):
					done_users.add(current_user)

			# Start a new message
			date_str, time_str, user, message = match.groups()
			current_user = user.strip()
			current_message = message.strip()

			# Convert message date to a date object and check if within date range
			message_date = datetime.strptime(date_str, date_format).date()

			if not (start_date <= message_date <= end_date):
				current_user = None  # Ignore this message if it's not in the date range
				current_message = ""
				continue

			# Check for added users or users added by someone
			added_match = added_pattern.search(current_message)
			added_by_match = added_by_pattern.search(current_message)

			if added_match:
				added_user = added_match.group(1).strip()
				added_users.add(added_user)

			if added_by_match:
				added_by = added_by_match.group(1).strip()
				added_user = added_by_match.group(2).strip()
				added_users.add(added_user)
				added_by_others.add(added_by)
				
			elif join_match:
				# Process the "joined from the community" message
				date_str, time_str, user, _ = join_match.groups()
				user = user.strip()
				added_users.add(user)

			elif request_match:
				# Process the "requested to join" message
				date_str, time_str, user, _ = request_match.groups()
				user = user.strip()
				requested_to_join_users.add(user)
			

		else:
			# If no match, it means this is a continuation of the current message (multi-line)
			current_message += " " + line.strip()

	# Process the last message after the loop ends
	if current_user and current_message:
		cleaned_message = current_message.strip().lower()
		if ('done' in cleaned_message or 
			'dn' in cleaned_message or 
			'✅' in cleaned_message or
			'done✅' in cleaned_message or
			'done' in cleaned_message or
			'تم ' in cleaned_message or
			'ضن' in cleaned_message or
			re.search(r'\bتم\b', cleaned_message) or 
			cleaned_message == 'تم' or 
			'تم' in cleaned_message):
			done_users.add(current_user)

	return done_users, added_users, added_by_others, requested_to_join_users


# Part 2: Compare Group Members with Task Completion

In [18]:
def compare_group_with_done_users_and_added(done_users, added_users, added_by_others, requested_to_join_users):
    """
    Comparing the list of groups that I got from 18.08.2024 (after the deadline one) until now.
    And find the user that didn't match with the existing user that send from 26.08.2024 (deadline 3)
    Also prints added users and who added them.
    """
    # Full list of group members provided
    group_members = {
"* PS",
". PS",
"A7 Hmd PS",
"Abdelhadi PS",
"Abdelrahman Karim",
"Ahmed Abdelsamie PS",
"Ahmed Elkassrawy PS",
"Ahmed Hesham",
"Ahmed Jamal Sultan PS",
"Ahmed Mohamed Badran",
"Ahmed Mounir PS",
"Ahmed Salman PS",
"Ali Ismail",
"Amal Eltelbany PS",
"Amany Mohammed PS",
"Amira PS",
"Asma 🪄",
"Asmaa",
"Ayash PS",
"Aysha",
"BΛSΛNT",
"David PS",
"Elknz PS",
"Eman Ashraf PS",
"Esraa",
"Esraa Alaa",
"Gehad PS",
"Gehad🌺",
"Habiba Gamal PS",
"Hadi Heikal PS",
"Hasnaa PS",
"Hoda Salim PS",
"Hüsam",
"Ichraf 🤓👩‍💻",
"KHALED𓂆",
"Karema PS",
"MGamal",
"MORo",
"Mahmoud",
"Mahmoud Elsheemy PS",
"Mai Samir PS",
"Manar PS",
"Mariam Osman PS",
"Mennatallah",
"Minno",
"Mohamed Rabee PS",
"Motaz PS",
"Nada Fahmy PS",
"Nada Osama",
"NourEldien Alsharif PS",
"Omnia Ashraf PS",
"Omnia PS",
"Omr PS",
"Os!Rabe3 PS",
"PS-StudyingAlgos 📚",
"Salah Mohamed Twtr",
"Sara",
"Shahd Tamer PS",
"Shaimaa",
"TURBO PS",
"Tarek",
"Wafaa",
"Yos S3D Twtr",
"Younes PS",
"Youssef Yousry PS",
"Zeyad",
"Zeyad Albadawy PS",
"Ziad Nasser PS",
"mahmoud elshony",
"أَدْهَم عَلَّاَم PS",
"رُفَيده سَعْد PS",
"رِيمُو PS",
"‎You",
"𓂆 PS",
"𝐈𝐬𝐥𝐚𝐦 𝐆𝐨𝐦𝐚𝐚",
"𝓐𝓁𝒶𝒶 PS",
}


    # Find users in the group who have not completed the task
    not_done_users = group_members - done_users

    # Output users who did NOT complete the task
    print("\nUsers who did NOT complete the task:")
    if not_done_users:
        for user in sorted(not_done_users):
            print(user)
    else:
        print("All users completed the task.")

    print("\n==================================")
    print("\nUsers who completed the task:")
    for user in sorted(done_users):
        print(user)

    # Output users who were added to the group
    print("\n==================================")
    print("\nUsers who were added to the group:")
    for user in sorted(added_users):
        print(user)

    # Output users who added others to the group
    print("\n==================================")
    print("\nUsers who added someone to the group:")
    for user in sorted(added_by_others):
        print(user)
        
	# Output users who requested to join the group
    print("\n==================================")
    print("\nUsers who requested to join the group:")
    for user in sorted(requested_to_join_users):
        print(user)


# Part 3: Main Function

In [17]:
def main():
    """Main Function to apply all the functions written above"""
    # File path of exported WhatsApp chat (change it to your file's location)
    file_path = '_chat_5_algo.txt'

    # Specify the date range (start date and end date)
    start_date = '13.09.2024'
    end_date = date.today().strftime('%d.%m.%Y')  # Use today's date

    # Extract users who completed the task, users who were added, and who requested to join
    done_users, added_users, added_by_others, requested_to_join_users = parse_task_completion_and_additions(file_path, start_date, end_date)

    # Compare the task completion with the full list of group members and output added users and who requested to join
    compare_group_with_done_users_and_added(done_users, added_users, added_by_others, requested_to_join_users)

if __name__ == '__main__':
    main()


Users who did NOT complete the task:
* PS
. PS
Abdelrahman Karim
Ahmed Jamal Sultan PS
Ahmed Mohamed Badran
Amal Eltelbany PS
Asmaa
Ayash PS
Aysha
BΛSΛNT
David PS
Esraa Alaa
Gehad PS
Gehad🌺
Habiba Gamal PS
Hadi Heikal PS
Ichraf 🤓👩‍💻
KHALED𓂆
MGamal
MORo
Mahmoud
Minno
Nada Osama
Omnia Ashraf PS
Omnia PS
Omr PS
PS-StudyingAlgos 📚
Sara
Wafaa
Ziad Nasser PS
mahmoud elshony
أَدْهَم عَلَّاَم PS
رِيمُو PS
‎You
𝐈𝐬𝐥𝐚𝐦 𝐆𝐨𝐦𝐚𝐚


Users who completed the task:
A7 Hmd PS
Abdelhadi PS
Ahmed Abdelsamie PS
Ahmed Elkassrawy PS
Ahmed Hesham
Ahmed Mounir PS
Ahmed Salman PS
Ali Ismail
Amany Mohammed PS
Amira PS
Asma 🪄
Elknz PS
Eman Ashraf PS
Esraa
Hasnaa PS
Hoda Salim PS
Hüsam
Karema PS
Mahmoud Elsheemy PS
Mai Samir PS
Manar PS
Mariam Osman PS
Mennatallah
Mohamed Rabee PS
Motaz PS
Nada Fahmy PS
NourEldien Alsharif PS
Os!Rabe3 PS
Salah Mohamed Twtr
Shahd Tamer PS
Shaimaa
TURBO PS
Tarek
Yos S3D Twtr
Younes PS
Youssef Yousry PS
Zeyad
Zeyad Albadawy PS
رُفَيده سَعْد PS
𓂆 PS
𝓐𝓁𝒶𝒶 PS


Users who were added to the

# PART4: Generate a list of Group-members 
### `(with a specific date until now)`

In [14]:
def extract_group_members(file_path, start_date, end_date):
    # Set to store unique users
    group_members = set()

    # Read the WhatsApp export file
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Specify the date format that matches your WhatsApp's export format (dd.mm.yyyy)
    date_format = '%d.%m.%Y'

    # Convert the start and end dates to datetime objects
    start_date = datetime.strptime(start_date, date_format).date()
    end_date = datetime.strptime(end_date, date_format).date()

    # Regex pattern to capture WhatsApp message with date, time, user, and message
    # pattern = re.compile(r'\[(\d{1,2}\.\d{1,2}\.\d{4}) (\d{2}:\d{2}:\d{2})\] (?:~\s*|)(.*?): (.*)')
    pattern = re.compile(r'\[(\d{1,2}\.\d{1,2}\.\d{4})(?:,)? (\d{2}:\d{2}:\d{2})\] (?:~\s*|)(.*?): (.*)')

    for line in lines:
        match = pattern.match(line)
        if match:
            date_str, time_str, user, message = match.groups()

            # Convert message date to a date object
            message_date = datetime.strptime(date_str, date_format).date()

            # Check if the message date is within the specified date range
            if start_date <= message_date <= end_date:
                # Clean up the user name by stripping extra spaces or non-visible characters
                user = user.strip()

                # Debugging: Print the users being processed
                print(f"Processing message from: {user} at {date_str}")

                group_members.add(user)
        else:
            # Log lines that don't match to debug why certain messages are missed
            print(f"Line not matched: {line.strip()}")

    return group_members

def main():
    # File path of exported WhatsApp chat (change it to your file's location)
    file_path = '_chat_5_algo.txt'

    # Specify the date range (start date and end date)
    start_date = '18.08.2024'
    end_date = date.today().strftime('%d.%m.%Y')  # Use today's date

    # Extract group members who sent messages between the start and end date
    group_members = extract_group_members(file_path, start_date, end_date)

    # Output the group members
    print(f"\nGroup members who sent messages between {start_date} and {end_date}:")
    if group_members:
        for member in sorted(group_members):
            print(f'"{member}",')
    else:
        print("No messages found in the given date range.")

if __name__ == '__main__':
    main()


Line not matched: هنمشيها ازاي مذاكرة algo
Line not matched: ‎OPTION: اسبوع اسبوع (15 votes)
Line not matched: ‎OPTION: نحط خطة شهرية (0 votes)
Line not matched: ابعت الpoll تاني كدا
Line not matched: مذاكرة algo
Line not matched: ‎OPTION: نحط بلان اسبوعيا (4 votes)
Line not matched: ‎OPTION: بلان شهريا (1 vote)
Line not matched: ‎OPTION: بلان اسبوعي + شهري (17 votes)
Line not matched: ‎[6.04.2024, 05:27:15] Hüsam: ‎sticker omitted
Line not matched: هنذاكرو من هنا ؟
Line not matched: تيلجرام احسن تطبيق في حاجات بتاع المجتمعات ومنظم اكتر
Line not matched: وبعد العيد جد الجد
Line not matched: وكورس التطبيق على الخوارزميات بتاع عادل نسيم هتلاقيه على جروب ال resources هو DS برضه
Line not matched: 
Line not matched: https://youtube.com/playlist?list=PL1DUmTEdeA6JlommmGP5wicYLxX5PVCQt&si=Yrg68XX0TITSMiD0
Line not matched: ال عايز يبعت ريسورس يعملي منشن او يبعتلي برايفيت مسدج اللينك مع كلمتين شرح ايه الريسورس وبيشرح ايه بس
Line not matched: وشكرا
Line not matched: ‎[6.04.2024, 05:36:31] ~ ريم